# Projet : Traitement Automatique des Langues (Partie 1)

Nous avons à notre disposition deux fichiers CSV ([allocine_genres_test.csv](data/allocine_genres_test.csv) et [allocine_genres_train.csv](data/allocine_genres_train.csv)) contenant des informations sur des films et leurs genres. Le but de ce projet est de prédire les genres d'un film à partir de son synopsis notamment (et d'autres informations).

L’objectif est d’entraîner un outil de classification automatique des films en fonction de leur genre. La classification doit se baser sur le texte de la synopsis et sur le titre des films. Le texte et le titre des articles ont déjà été tokenisés et tous les tokens sont séparés par un espace.

## 1. Importation des données et analyse exploratoire

Les données sont disponibles dans le dossier [data](data/). Nous allons commencer par importer les données et les analyser.

In [1]:
%%capture

# exécuter cette cellule pour installer les dépendances et télécharger les modèles spacy
# remplacer `python` par `python3` si nécessaire
!python -m pip install --upgrade -r requirements.txt
!python -m spacy download fr_core_news_sm

In [2]:
import os
import sys

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
import spacy

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from spacy import displacy

train = pd.read_csv(os.path.join('data', 'allocine_genres_train.csv'))

In [3]:
print(train.shape)
print(train.head())

(2875, 22)
   Unnamed: 0         acteur_1           acteur_2           acteur_3   
0        4772    Albert Finney      Lauren Bacall  Jacqueline Bisset  \
1         335      Henry Fonda      Martin Balsam       John Fiedler   
2        4860   Alexandra Lamy  Michaël Abiteboul       Julia Piaton   
3        1913  Charles Chaplin  Virginia Cherrill        Harry Myers   
4        3726   Robert De Niro   Cuba Gooding Jr.    Charlize Theron   

   allocine_id  annee_prod  annee_sortie  box_office_fr  couleur  duree  ...   
0         1453        1974        1975.0       549055.0  Couleur  128.0  ...  \
1         4063        1957        1957.0            NaN      NaN   95.0  ...   
2       241952        2016        2019.0            NaN  Couleur   90.0  ...   
3         2256        1931        1931.0            NaN      NaN   87.0  ...   
4        27434        2000        2001.0       221677.0  Couleur  129.0  ...   

  nb_critiques_presse nb_critiques_spectateurs  nb_notes_spectateurs   
0  

In [4]:
print(train['genre'].value_counts(), end='\n\n')
print(random.choice(train['synopsis'].unique()))

genre
drame              501
comédie            483
romance            443
policier           331
horreur            299
science fiction    298
biopic             191
documentaire       167
historique         162
Name: count, dtype: int64

On dit de Lou qu’ elle est une enfant précoce . Elle a treize ans , deux classes d’ avance et un petit corps qui prend son temps . Elle a une mère emmurée dans les tranquillisants , peu d’ amis , et le ressenti aigu d’ un monde qui va de travers . Lou doit faire un exposé sur une jeune femme sans abri . Elle en a vu une à la Gare d’ Austerlitz . Une qui fait la manche , demande des clopes , s’ endort sur la table du café lorsque Lou lui offre à boire pour l’ interviewer . Elle a 18 ans , s’ appelle No , Nora en fait mais tout le monde dit No , et bientôt Lou ne pourra plus se passer d’ elle . Mais No est imprévisible , elle a grandi dans les foyers et elle ne ressemble à personne . Un jour , elle disparaît . Lou la recherche , sûre de ce besoin qu’ e

On remarque déjà que les donnés chiffrées sont soit des entiers soit des flottants. Les données textuelles sont des chaînes de caractères. Certaines données sont manquantes : `NaN` dans le cas des données chiffrées et une chaîne vide dans le cas des données textuelles.

Pour traiter les données manquantes, nous avons deux solutions :

1. Dans le cas où il y a très peu de données manquantes, on peut simplement supprimer les entrées qui contiennent ces données manquantes. Cela peut être acceptable si le nombre de données manquantes est très faible par rapport à la taille de l'ensemble de données et que la suppression de ces entrées n'affecte pas significativement les résultats de l'analyse.
2. En revanche, si le nombre de données manquantes est important, la suppression de ces entrées pourrait entraîner une perte d'informations importantes pour l'analyse. Dans ce cas, il est généralement préférable de remplacer les valeurs manquantes par une valeur qui représente au mieux l'information manquante. Par exemple, si les données manquantes sont des scores au box-office, vous pouvez remplacer ces données manquantes par la moyenne ou la médiane des scores de box-office disponibles dans les données.


In [5]:
EMPTY_TOKEN = '<EMPTY>'

# remove rows with missing values
train_dataset_1 = train.dropna(how='any', inplace=False)

# replace missing values with either the mean or the median (or empty token)
list_headers = train.columns.values.tolist()
list_of_numerical_headers = train._get_numeric_data().columns.values.tolist()
list_of_categorical_headers = list(set(list_headers) - set(list_of_numerical_headers))

train_dataset_2 = train.copy()
for header in list_of_numerical_headers:
  train_dataset_2[header].fillna(train_dataset_2[header].median(), inplace=True)
for header in list_of_categorical_headers:
  train_dataset_2[header].fillna(EMPTY_TOKEN, inplace=True)

print(train_dataset_1.shape)
print(train_dataset_2.shape)

(891, 22)
(2875, 22)


Lorsqu'on supprime simplement toutes les entrées où il manque au moins une valeur, on se retrouve uniquement avec 891 valeurs en tout. Cela signifie que nous avons perdu beaucoup d'informations. Nous allons donc utiliser la deuxième solution (au moins dans un premier temps) et remplacer les valeurs manquantes par des valeurs qui représentent au mieux l'information manquante.

## 2. Prétraitement des données

Il faut aussi corriger les entrées textuelles, ainsi qu'appliquer un certain nombre d'algorithmes de prétraitement comme : la suppression des caractères spéciaux, la suppression des stop words, la suppression des mots trop fréquents ou trop rares, la lemmatisation, la suppression des mots trop longs, etc.

On définit donc un ensemble de fonctions et de filtres qui vont nous permettre de prétraiter les données textuelles.

In [6]:
nltk.download('stopwords', quiet=True)        # download the stopwords corpus
nlp = spacy.load('fr_core_news_sm')           # load the French model
fr_stopwords = set(stopwords.words('french')) # so that `in` tests are faster
stemmer = SnowballStemmer(language='french')  # for stemming


# get the tokens of a sentence (word based tokenization)
def get_tokens_words(text: str) -> list[str]:
  doc = nlp(text)
  return [w.text for w in doc]


# remove stopwords from a sentence
def clean_sentence(text: str) -> list[str]:
  clean_words: list[str] = []
  for token in get_tokens_words(text):
    if token not in fr_stopwords:
      clean_words.append(token)
  return clean_words


# get the tokens of multiple sentences (sentence based tokenization)
def get_tokens_sentences(text: str) -> list[str]:
  doc = nlp(text)
  return [s.text for s in doc.sents]


# get the lemmas of a sentence
def get_stem(text: str) -> list[str]:
  doc = nlp(text)
  return [stemmer.stem(w.text) for w in doc]


# get the named entities of a sentence
def get_ner(text: str) -> list[str]:
  doc = nlp(text)
  return [(ent.text, ent.label_) for ent in doc.ents]


# render the named entities of a sentence in a Jupyter notebook
def render_ner(text: str) -> None:
  doc = nlp(text)
  displacy.render(doc, style='ent', jupyter=True)


# get the part of speech of a sentence
def get_pos(text: str) -> list[str]:
  doc = nlp(text)
  return [(token, token.pos_) for token in doc]


# render the part of speech of a sentence in a Jupyter notebook
def render_pos(text: str) -> None:
  doc = nlp(text)
  displacy.render(doc, style='dep', options={'distance': 90})


# get the word embeddings of a sentence
def get_word_embeddings(text: str) -> list[np.ndarray]:
  doc = nlp(text)
  return [token.vector for token in doc]


# get the similarity between two sentences
def get_mean_embedding(text1: str, text2: str) -> float:
  doc1 = nlp(text1)
  doc2 = nlp(text2)
  mean1 = np.mean([token.vector for token in doc1], axis=0)
  mean2 = np.mean([token.vector for token in doc2], axis=0)

  return np.dot(mean1, mean2) / (np.linalg.norm(mean1) * np.linalg.norm(mean2))

Juste pour clarifier les choses, nous allons simplement effectuer des tests sur l'ensemble de phrases suivantes :

1. "Le réseau sera bientôt rétabli à Marseille"
2. "La panne réseau affecte plusieurs utilisateurs de l'opérateur"
3. "Il fait 18 degrés ici"
4. "Bouygues a eu une coupure de réseau à Marseille. La panne a affecté 300.000 utilisateurs."

In [7]:
text1 = 'Le réseau sera bientôt rétabli à Marseille.'
text2 = 'La panne réseau affecte plusieurs utilisateurs de l\'opérateur'
text3 = 'Il fait 18 degrés ici'
text4 = 'Bouygues a eu une coupure de réseau à Marseille. La panne a affecté 300.000 utilisateurs.'

In [8]:
# basic tokenization
# we can observe `get_tokens_sentences` do not "cut" at each . or ! or ?

print(get_tokens_words(text1))
print(clean_sentence(text1))
print(get_tokens_sentences(text4))

['Le', 'réseau', 'sera', 'bientôt', 'rétabli', 'à', 'Marseille', '.']
['Le', 'réseau', 'bientôt', 'rétabli', 'Marseille', '.']
['Bouygues a eu une coupure de réseau à Marseille.', 'La panne a affecté 300.000 utilisateurs.']


In [9]:
# stemming
# this doesn't work very well for French...

print(get_stem(text1))
print(get_stem(text2))

['le', 'réseau', 'ser', 'bientôt', 'rétabl', 'à', 'marseil', '.']
['la', 'pann', 'réseau', 'affect', 'plusieur', 'utilis', 'de', "l'", 'oper']


In [10]:
# named entities recognition

print(get_ner(text4))
render_ner(text4)

[('Bouygues', 'ORG'), ('Marseille', 'LOC')]


In [11]:
# part of speech

print(get_pos(text1))
render_pos(text1)

[(Le, 'DET'), (réseau, 'NOUN'), (sera, 'AUX'), (bientôt, 'ADV'), (rétabli, 'ADJ'), (à, 'ADP'), (Marseille, 'PROPN'), (., 'PUNCT')]


In [12]:
# word embeddings and mean embedding (similarity)

print(get_word_embeddings(text1)[0].shape)
print(get_mean_embedding(text1, text2))
print(get_mean_embedding(text1, text4))
print(get_mean_embedding(text2, text4))

(96,)
0.29973912
0.39468837
0.53404236
